### <a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Brooklyn and Queens</font></h1>


## Introduction

A Texas based chain of Carribean restaurants is planning to expand in Brooklyn and Queens boroughs of New York City.  They have approached a data scientist to analyze which neighborhood(s) of these boroughs have heavier footfall as far as Carribean food is concerned.  

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in New York City</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Download all the dependencies that are needed for the project.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


### Download data for New York City.  Use the same file as provided in the lab for the course.


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

    
print('Data downloaded!')

Data downloaded!


#### Load and explore the data


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    

Let's take a quick look at the data.


### newyork_data

All the relevant data is in the _features_ key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.


In [6]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.


In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a _pandas_ dataframe


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.


In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.


In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.


In [11]:
neighborhoods.head(30)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.


In [14]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

For this project we will segment and cluster the neighborhoods in Brooklyn and Queens separately. So let's slice the original dataframe and create  new dataframes of the Brooklyn and Queens data.


In [15]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [16]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Let's get the geographical coordinates of Brooklyn and Queens.


In [17]:
address_brk = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_brk)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))


address_qns = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_qns)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.
The geograpical coordinate of Queens are 40.7498243, -73.7976337.


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [18]:
CLIENT_ID = 'XXXXXXXXX' # Foursquare ID faked to maintain my privacy
CLIENT_SECRET = 'XXXXXXXX' # Foursquare Secret faked to maintain my privacy
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H3CCWLNT0K2MSPJSESZIGEI205LEZRNCGZVZ2R502IYVK1PA
CLIENT_SECRET:3M1NFKQGWS1W0TYLELBHIZ5PE4TCEAAHR4PIJBTBEEIVZX0V


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


brooklyn_data.loc[0, 'Neighborhood']

Get the neighborhood's latitude and longitude values.


neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

<a id='item2'></a>


## 2. Explore Neighborhoods in Brooklyn and Queens


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create new dataframes called brooklyn_venues  and queens_venues.


In [21]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [22]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


#### Let's check the size of the resulting dataframes


In [29]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(7000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
1,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
2,Bay Ridge,40.625801,-74.030621,Mimi Nails,40.622571,-74.031477,Spa
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Little Cupcake Bakeshop,40.620021,-74.032567,Cupcake Shop


In [30]:
print(queens_venues.shape)
queens_venues.head()

(8006, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
1,Astoria,40.768509,-73.915654,Ample Hills Creamery,40.765562,-73.919012,Ice Cream Shop
2,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
3,Astoria,40.768509,-73.915654,Al-sham Sweets and Pastries,40.768077,-73.911561,Middle Eastern Restaurant
4,Astoria,40.768509,-73.915654,Brooklyn Bagel & Coffee Co.,40.764732,-73.916944,Bagel Shop


Let's check how many venues were returned for each neighborhood


In [31]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,100,100,100,100,100,100
Bay Ridge,100,100,100,100,100,100
Bedford Stuyvesant,100,100,100,100,100,100
Bensonhurst,100,100,100,100,100,100
Bergen Beach,100,100,100,100,100,100
Boerum Hill,100,100,100,100,100,100
Borough Park,100,100,100,100,100,100
Brighton Beach,100,100,100,100,100,100
Broadway Junction,100,100,100,100,100,100


In [32]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,84,84,84,84,84,84
Astoria,100,100,100,100,100,100
Astoria Heights,100,100,100,100,100,100
Auburndale,100,100,100,100,100,100
Bay Terrace,100,100,100,100,100,100
Bayside,100,100,100,100,100,100
Bayswater,100,100,100,100,100,100
Beechhurst,100,100,100,100,100,100
Bellaire,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues


In [33]:
print('There are {} uniques categories for Brooklyn.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 230 uniques categories for Brooklyn.


In [34]:
print('There are {} uniques categories for Queens.'.format(len(queens_venues['Venue Category'].unique())))

There are 259 uniques categories for Queens.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [35]:
# one hot encoding for Brooklyn
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Picnic Shelter,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,State / Provincial Park,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tibetan Restaurant,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yemeni Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [36]:
# one hot encoding for queens
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,Hookah Bar,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern Greek Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Reservoir,Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,State / Provincial Park,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

And let's examine the new dataframe size.


In [37]:
brooklyn_onehot.shape

(7000, 230)

In [38]:
queens_onehot.shape

(8006, 259)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [39]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Picnic Shelter,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stables,State / Provincial Park,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tibetan Restaurant,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yemeni Restaurant
0,Bath Beach,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.07,0.00,0.03,0.00,0.01,0.00,0.03,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.03,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.06,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0

In [40]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,Hookah Bar,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern Greek Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Record Shop,Rental Car Location,Reservoir,Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,State / Provincial Park,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Arverne,0.00,0.000000,0.00,0.00,0.000000,0.00,0.011905,0.00,0.011905,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.071429,0.023810,0.000000,0.011905,0.000000,0.00,0.000000,0.345238,0.023810,0.00,0.00,0.00,0.00,0.00,0.00,0.0000

#### Let's confirm the new size


In [41]:
brooklyn_grouped.shape

(70, 230)

In [42]:
queens_grouped.shape

(81, 259)

#### Let's print each neighborhood along with the top 5 most common venues


num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframes and display the top 10 venues for each neighborhood.


In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe for Brooklyn
brkneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
brkneighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brkneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brkneighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pizza Place,Bakery,Italian Restaurant,Scenic Lookout,Beach,Bar,Bubble Tea Shop,Spa,Bagel Shop,Coffee Shop
1,Bay Ridge,Pizza Place,Bar,Greek Restaurant,Italian Restaurant,Bakery,Bagel Shop,Gym / Fitness Center,Coffee Shop,Grocery Store,Park
2,Bedford Stuyvesant,Bar,Pizza Place,Italian Restaurant,Tapas Restaurant,Gourmet Shop,Coffee Shop,Brewery,New American Restaurant,Cocktail Bar,Farmers Market
3,Bensonhurst,Pizza Place,Bakery,Italian Restaurant,Bar,Scenic Lookout,Park,Spa,Grocery Store,Sushi Restaurant,Gym / Fitness Center
4,Bergen Beach,Pizza Place,Pharmacy,Bagel Shop,Bakery,Deli / Bodega,Italian Restaurant,Gym,Chinese Restaurant,Caribbean Restaurant,Coffee Shop


In [45]:
# create a new dataframe for Queens
qnsneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
qnsneighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    qnsneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

qnsneighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Beach,Bagel Shop,Surf Spot,Pizza Place,Italian Restaurant,Bakery,Wine Bar,Burger Joint,Beach Bar,Chinese Restaurant
1,Astoria,Bar,Italian Restaurant,Greek Restaurant,Pizza Place,Cocktail Bar,Bakery,Thai Restaurant,Ice Cream Shop,Coffee Shop,Bagel Shop
2,Astoria Heights,Greek Restaurant,Bar,Bakery,Bagel Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Coffee Shop,Food Truck,Indian Restaurant
3,Auburndale,Pizza Place,Korean Restaurant,Park,Bakery,Indian Restaurant,Greek Restaurant,Italian Restaurant,Bagel Shop,Hotpot Restaurant,Coffee Shop
4,Bay Terrace,Pizza Place,Bakery,Korean Restaurant,Greek Restaurant,Deli / Bodega,Park,Italian Restaurant,Ice Cream Shop,Bagel Shop,Indian Restaurant


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [46]:
# set number of clusters for Brooklyn
brkkclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
brkkmeans = KMeans(n_clusters=brkkclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
brkkmeans.labels_[0:10] 

array([4, 0, 2, 0, 0, 2, 0, 4, 3, 2], dtype=int32)

In [47]:
# set number of clusters for Queens
qnskclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
qnskmeans = KMeans(n_clusters=qnskclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
qnskmeans.labels_[0:10] 

array([1, 4, 4, 2, 2, 2, 1, 2, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [48]:
# add clustering labels for Brooklyn
brkneighborhoods_venues_sorted.insert(0, 'Cluster Labels', brkkmeans.labels_)

brooklyn_merged = brooklyn_data

# merge brooklyn_grouped with brooklyn_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(brkneighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head(100) 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Pizza Place,Bar,Greek Restaurant,Italian Restaurant,Bakery,Bagel Shop,Gym / Fitness Center,Coffee Shop,Grocery Store,Park
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0,Pizza Place,Bakery,Italian Restaurant,Bar,Scenic Lookout,Park,Spa,Grocery Store,Sushi Restaurant,Gym / Fitness Center
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Pizza Place,Park,Mexican Restaurant,Brewery,Bakery,Wine Shop,Coffee Shop,Food & Drink Shop,Spa,Italian Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Gourmet Shop,Pizza Place,Bakery,Japanese Restaurant,Park,Cocktail Bar,Ice Cream Shop,Café,Wine Shop,Brewery
4,Brooklyn,Gravesend,40.595260,-73.973471,4,Pizza Place,Bagel Shop,Beach,Bakery,Italian Restaurant,Sandwich Place,Grocery Store,Turkish Restaurant,Mediterranean Restaurant,Sushi Restaurant
5,Brooklyn,Brighton Beach,40.576825,-73.965094,4,Pizza Place,Beach,Theme Park Ride / Attraction,Sandwich Place,Bagel Shop,Italian Restaurant,Grocery Store,Turkish Restaurant,Bar,Bakery
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,4,Pizza Place,Bagel Shop,Grocery Store,Beach,Restaurant,Japanese Restaurant,Sandwich Place,Theme Park Ride / Attraction,Mediterranean Restaurant,Sushi Restaurant
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,0,Pizza Place,Italian Restaurant,Bakery,Café,Sandwich Place,Bagel Shop,Sushi Restaurant,Bar,Ice Cream Shop,Gym / Fitness Center
8,Brooklyn,Flatbush,40.636326,-73.958401,1,Pizza Place,Park,Café,Garden,Caribbean Restaurant,Italian Restaurant,Chinese Restaurant,Bakery,Wine Shop,American Restaurant
9,Brooklyn,Crown Heights,40.670829,-73.943291,1,Bar,Park,Cocktail Bar,Italian Restaurant,Pizza Place,Coffee Shop,Performing Arts Venue,Bookstore,Café,Gym


In [49]:
# add clustering labels for Queens
qnsneighborhoods_venues_sorted.insert(0, 'Cluster Labels', qnskmeans.labels_)

queens_merged = queens_data

# merge queens_grouped with queens_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(qnsneighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head(100) 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,4,Bar,Italian Restaurant,Greek Restaurant,Pizza Place,Cocktail Bar,Bakery,Thai Restaurant,Ice Cream Shop,Coffee Shop,Bagel Shop
1,Queens,Woodside,40.746349,-73.901842,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Queens,Jackson Heights,40.751981,-73.882821,4,Thai Restaurant,Bakery,Pizza Place,Bar,Tennis Stadium,Bagel Shop,Park,South American Restaurant,Ice Cream Shop,Martial Arts School
3,Queens,Elmhurst,40.744049,-73.881656,4,Thai Restaurant,Pizza Place,Bakery,Tennis Stadium,Bagel Shop,South American Restaurant,Park,Italian Restaurant,Farmers Market,Martial Arts School
4,Queens,Howard Beach,40.654225,-73.838138,0,Pizza Place,Discount Store,Caribbean Restaurant,Restaurant,Pharmacy,Italian Restaurant,Bakery,Burger Joint,Gym,Department Store
5,Queens,Corona,40.742382,-73.856825,4,Tennis Stadium,Thai Restaurant,Pizza Place,South American Restaurant,Chinese Restaurant,Burger Joint,Bakery,Italian Restaurant,Museum,Peruvian Restaurant
6,Queens,Forest Hills,40.725264,-73.844475,0,Pizza Place,Tennis Stadium,Bakery,Park,Burger Joint,Cosmetics Shop,Mediterranean Restaurant,Juice Bar,Italian Restaurant,Boxing Gym
7,Queens,Kew Gardens,40.705179,-73.829819,0,Pizza Place,Caribbean Restaurant,Bakery,Boxing Gym,Sushi Restaurant,South American Restaurant,Burger Joint,Food Truck,Bar,Bagel Shop
8,Queens,Richmond Hill,40.697947,-73.831833,0,Pizza Place,Caribbean Restaurant,Bakery,Burger Joint,Cosmetics Shop,Boxing Gym,Dog Run,Diner,Plaza,Bagel Shop
9,Queens,Flushing,40.764454,-73.831773,2,Tennis Stadium,Chinese Restaurant,Korean Restaurant,Park,Pizza Place,Hotpot Restaurant,Bakery,Bubble Tea Shop,Dumpling Restaurant,Coffee Shop


Finally, let's visualize the resulting clusters


In [53]:
# create map for Brooklyn Clusters
brkmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(brkkclusters)
ys = [i + x + (i*x)**2 for i in range(brkkclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.99).add_to(brkmap_clusters)
       
brkmap_clusters

In [54]:
# create map for Queens Clusters
qnsmap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(qnskclusters)
ys = [i + x + (i*x)**2 for i in range(brkkclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.99).add_to(qnsmap_clusters)
       
qnsmap_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Brooklyn Cluster 1


In [81]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Greenpoint,Gourmet Shop,Bakery,Japanese Restaurant,Cocktail Bar,Park,Café,Pizza Place,Wine Shop,Pier,Brewery
9,Crown Heights,Bar,Park,Cocktail Bar,Italian Restaurant,Coffee Shop,Pizza Place,Performing Arts Venue,Bookstore,Opera House,New American Restaurant
13,Prospect Heights,Bar,Cocktail Bar,Park,Pizza Place,Coffee Shop,Italian Restaurant,Performing Arts Venue,Bookstore,Farmers Market,Tapas Restaurant
15,Williamsburg,Gourmet Shop,Bakery,Bar,Park,Japanese Restaurant,Italian Restaurant,Ice Cream Shop,Pizza Place,Movie Theater,Juice Bar
17,Bedford Stuyvesant,Pizza Place,Bar,Italian Restaurant,New American Restaurant,Cocktail Bar,Coffee Shop,Brewery,Gourmet Shop,Bakery,Opera House
18,Brooklyn Heights,Park,Italian Restaurant,Ice Cream Shop,Memorial Site,Coffee Shop,Indie Movie Theater,Music Venue,Garden,Hotel,Thai Restaurant
19,Cobble Hill,Bar,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Memorial Site,Performing Arts Venue,Cocktail Bar,Pizza Place,Bookstore
20,Carroll Gardens,Bar,Italian Restaurant,Cocktail Bar,Park,Ice Cream Shop,Bookstore,Pizza Place,Performing Arts Venue,Coffee Shop,Brewery
21,Red Hook,Cocktail Bar,Park,Ice Cream Shop,Bar,Brewery,Coffee Shop,Pizza Place,Italian Restaurant,Bookstore,Yoga Studio
22,Gowanus,Bar,Cocktail Bar,Coffee Shop,Ice Cream Shop,Bookstore,Brewery,Park,Pizza Place,Italian Restaurant,Performing Arts Venue


#### Brooklyn Cluster 2


In [82]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Pizza Place,Greek Restaurant,Spa,Bakery,Bagel Shop,Italian Restaurant,Park,Scenic Lookout,Bar,Coffee Shop
1,Bensonhurst,Pizza Place,Bakery,Italian Restaurant,Bar,Spa,Park,Turkish Restaurant,Steakhouse,Sushi Restaurant,Tea Room
4,Gravesend,Pizza Place,Italian Restaurant,Beach,Sushi Restaurant,Bagel Shop,Sandwich Place,Gourmet Shop,Theme Park Ride / Attraction,Bubble Tea Shop,Mediterranean Restaurant
5,Brighton Beach,Restaurant,Beach,Sandwich Place,Italian Restaurant,Grocery Store,Pizza Place,Theme Park Ride / Attraction,Russian Restaurant,Bar,Turkish Restaurant
6,Sheepshead Bay,Restaurant,Pizza Place,Sushi Restaurant,Grocery Store,Bagel Shop,Beach,Sandwich Place,Russian Restaurant,Turkish Restaurant,Bar
7,Manhattan Terrace,Pizza Place,Italian Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Café,Russian Restaurant,Bar,Ice Cream Shop,Bagel Shop
31,Manhattan Beach,Beach,Restaurant,Sushi Restaurant,Bagel Shop,Grocery Store,Turkish Restaurant,Sandwich Place,Russian Restaurant,Bar,Pizza Place
32,Coney Island,Pizza Place,Italian Restaurant,Sushi Restaurant,Beach,Bagel Shop,Sandwich Place,Coffee Shop,Bubble Tea Shop,Grocery Store,Gourmet Shop
33,Bath Beach,Pizza Place,Italian Restaurant,Bakery,Spa,Bubble Tea Shop,Beach,Scenic Lookout,Bar,Theme Park Ride / Attraction,Mexican Restaurant
35,Dyker Heights,Pizza Place,Italian Restaurant,Spa,Bakery,Greek Restaurant,Scenic Lookout,Coffee Shop,Cosmetics Shop,Park,Gym / Fitness Center


#### Brooklyn Cluster 3


In [83]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Starrett City,Pizza Place,Italian Restaurant,Bakery,Breakfast Spot,Pharmacy,Ice Cream Shop,Bagel Shop,Supermarket,Caribbean Restaurant,Japanese Restaurant
28,Canarsie,Pizza Place,Caribbean Restaurant,Pharmacy,Discount Store,Bakery,Donut Shop,Deli / Bodega,Department Store,Diner,Breakfast Spot
29,Flatlands,Pizza Place,Caribbean Restaurant,Café,Pharmacy,Coffee Shop,Bakery,Italian Restaurant,Liquor Store,Deli / Bodega,Mediterranean Restaurant
30,Mill Island,Pizza Place,Bagel Shop,Pharmacy,Italian Restaurant,Sushi Restaurant,Park,Deli / Bodega,American Restaurant,Donut Shop,Turkish Restaurant
37,Marine Park,Pizza Place,Bagel Shop,Pharmacy,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Donut Shop,Ice Cream Shop,Turkish Restaurant,Park
45,Bergen Beach,Pizza Place,Pharmacy,Bagel Shop,Caribbean Restaurant,Bakery,Deli / Bodega,Italian Restaurant,Sandwich Place,Chinese Restaurant,Event Space
48,Georgetown,Pizza Place,Pharmacy,Caribbean Restaurant,Bagel Shop,Italian Restaurant,Bakery,Donut Shop,Discount Store,Deli / Bodega,American Restaurant
59,Paerdegat Basin,Caribbean Restaurant,Pizza Place,Pharmacy,Bakery,Discount Store,Donut Shop,Italian Restaurant,Breakfast Spot,Burger Joint,Chinese Restaurant
60,Mill Basin,Pizza Place,Pharmacy,Bagel Shop,Caribbean Restaurant,Italian Restaurant,Park,Donut Shop,Deli / Bodega,American Restaurant,Sandwich Place


#### Brooklyn Cluster 4


In [84]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Pizza Place,Park,Mexican Restaurant,Brewery,Food & Drink Shop,Italian Restaurant,Coffee Shop,Spa,Bar,Wine Shop
8,Flatbush,Café,Park,Pizza Place,Garden,Caribbean Restaurant,Italian Restaurant,Bakery,Chinese Restaurant,Wine Shop,American Restaurant
10,East Flatbush,Café,Pizza Place,Caribbean Restaurant,Park,Playground,Garden,Bakery,Coffee Shop,Thai Restaurant,Restaurant
11,Kensington,Park,Brewery,Coffee Shop,Bar,Pizza Place,Caribbean Restaurant,Italian Restaurant,Garden,Bakery,Café
12,Windsor Terrace,Brewery,Park,Pizza Place,Bar,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Cocktail Bar,Ice Cream Shop,Bakery
34,Borough Park,Pizza Place,Italian Restaurant,Park,Coffee Shop,Bakery,Mexican Restaurant,American Restaurant,Brewery,Bar,Café
42,Prospect Lefferts Gardens,Cocktail Bar,Bar,Caribbean Restaurant,Park,Pizza Place,Café,Bakery,Gift Shop,Coffee Shop,Beer Bar
46,Midwood,Pizza Place,Café,Italian Restaurant,Caribbean Restaurant,Park,Thai Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Chinese Restaurant
47,Prospect Park South,Bar,Caribbean Restaurant,Pizza Place,Park,Coffee Shop,Café,Mexican Restaurant,Garden,Italian Restaurant,Cocktail Bar
54,Ditmas Park,Bar,Pizza Place,Caribbean Restaurant,Coffee Shop,Café,Park,Garden,Italian Restaurant,Mexican Restaurant,Yoga Studio


#### Brooklyn Cluster 5


In [85]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Brownsville,Bar,Bakery,Caribbean Restaurant,Pizza Place,Coffee Shop,French Restaurant,Park,Cocktail Bar,Juice Bar,American Restaurant
16,Bushwick,Pizza Place,Bar,Bakery,Coffee Shop,Brewery,Gourmet Shop,Italian Restaurant,Café,Sandwich Place,New American Restaurant
25,Cypress Hills,Pizza Place,Bar,Coffee Shop,Bakery,Park,Brewery,Caribbean Restaurant,Grocery Store,Nightclub,German Restaurant
26,East New York,Coffee Shop,Pizza Place,Bakery,Caribbean Restaurant,Bar,Park,Brewery,Nightclub,Ice Cream Shop,Italian Restaurant
43,Ocean Hill,Bar,Pizza Place,Caribbean Restaurant,Bakery,Coffee Shop,Cocktail Bar,Italian Restaurant,Nightclub,Mexican Restaurant,New American Restaurant
44,City Line,Pizza Place,Bakery,Bar,Coffee Shop,Grocery Store,Deli / Bodega,Nightclub,Burger Joint,Discount Store,Brewery
58,New Lots,Bakery,Coffee Shop,Pizza Place,Bar,Park,Italian Restaurant,Caribbean Restaurant,Breakfast Spot,Brewery,Ice Cream Shop
64,Broadway Junction,Bar,Pizza Place,Bakery,Nightclub,Mexican Restaurant,Coffee Shop,Italian Restaurant,Park,Caribbean Restaurant,Sandwich Place
67,Highland Park,Bar,Coffee Shop,Bakery,Pizza Place,Brewery,Mexican Restaurant,Nightclub,Grocery Store,Park,Gym


#### Queens Cluster 1

In [90]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Howard Beach,Pizza Place,Caribbean Restaurant,Ice Cream Shop,Italian Restaurant,Discount Store,Gym,Department Store,Pet Store,Pharmacy,Restaurant
6,Forest Hills,Pizza Place,Tennis Stadium,Bakery,Park,Burger Joint,Bagel Shop,Italian Restaurant,South American Restaurant,Ice Cream Shop,Seafood Restaurant
7,Kew Gardens,Pizza Place,Caribbean Restaurant,Bakery,Burger Joint,Boxing Gym,Plaza,Sushi Restaurant,Bagel Shop,Seafood Restaurant,Cosmetics Shop
8,Richmond Hill,Pizza Place,Caribbean Restaurant,Bakery,Burger Joint,Plaza,Dog Run,Sushi Restaurant,Juice Bar,Thai Restaurant,Diner
15,Glendale,Pizza Place,Bakery,Park,Bar,Grocery Store,Nightclub,Brewery,American Restaurant,Burger Joint,Coffee Shop
16,Rego Park,Pizza Place,Tennis Stadium,Park,Bakery,Thai Restaurant,Burger Joint,Mediterranean Restaurant,Seafood Restaurant,Boxing Gym,Coffee Shop
17,Woodhaven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Ozone Park,Pizza Place,Caribbean Restaurant,Diner,Ice Cream Shop,Bakery,Italian Restaurant,Deli / Bodega,Gym,Pet Store,Burger Joint
26,Glen Oaks,Pizza Place,Italian Restaurant,Grocery Store,Diner,Coffee Shop,Gym,Burger Joint,Deli / Bodega,Bakery,Bagel Shop
27,Bellerose,Pizza Place,Bakery,Park,Italian Restaurant,Bagel Shop,Diner,Deli / Bodega,Indian Restaurant,Ice Cream Shop,Gym / Fitness Center


#### Queens Cluster 2

In [91]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Far Rockaway,Beach,Coffee Shop,Golf Course,Ice Cream Shop,Pizza Place,Bagel Shop,Gym / Fitness Center,Italian Restaurant,Supermarket,Sandwich Place
42,Broad Channel,Beach,Bagel Shop,Donut Shop,Surf Spot,Pub,Bakery,Ice Cream Shop,Pizza Place,Coffee Shop,Burger Joint
47,Edgemere,Beach,Bagel Shop,Pizza Place,Surf Spot,Gym / Fitness Center,Chinese Restaurant,Burger Joint,Coffee Shop,Beach Bar,Bakery
48,Arverne,Beach,Bagel Shop,Surf Spot,Burger Joint,Coffee Shop,Chinese Restaurant,Bakery,Italian Restaurant,Beach Bar,Deli / Bodega
49,Rockaway Beach,Beach,Bagel Shop,Surf Spot,Donut Shop,Seafood Restaurant,Burger Joint,Coffee Shop,Beach Bar,Pizza Place,Bakery
50,Neponsit,Beach,Bagel Shop,Bar,Trail,Park,Bakery,Wine Bar,Donut Shop,Golf Course,Seafood Restaurant
61,Belle Harbor,Beach,Bagel Shop,Bar,Bakery,Beach Bar,Seafood Restaurant,Bus Stop,Trail,Surf Spot,Burger Joint
62,Rockaway Park,Beach,Bagel Shop,Surf Spot,National Park,Seafood Restaurant,Park,Wine Bar,Bakery,Donut Shop,Bar
63,Somerville,Beach,Pizza Place,Surf Spot,Donut Shop,Bagel Shop,Burger Joint,Ice Cream Shop,Beach Bar,Supermarket,Wine Bar
75,Roxbury,Beach,Bagel Shop,Donut Shop,Pizza Place,National Park,Bar,Bank,Turkish Restaurant,Scenic Lookout,Sandwich Place


#### Queens Cluster 3

In [92]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Flushing,Korean Restaurant,Chinese Restaurant,Tennis Stadium,Bubble Tea Shop,Park,Dumpling Restaurant,Pizza Place,Hotpot Restaurant,Bakery,Deli / Bodega
20,College Point,Park,Bubble Tea Shop,Pizza Place,Korean Restaurant,Tennis Stadium,Chinese Restaurant,Hotpot Restaurant,Deli / Bodega,Dumpling Restaurant,Dessert Shop
21,Whitestone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Bayside,Bakery,Korean Restaurant,Park,Pizza Place,Greek Restaurant,Italian Restaurant,Bagel Shop,Supermarket,Cosmetics Shop,Coffee Shop
23,Auburndale,Pizza Place,Korean Restaurant,Bakery,Park,Greek Restaurant,Indian Restaurant,Italian Restaurant,Bagel Shop,Hotpot Restaurant,Supermarket
24,Little Neck,Pizza Place,Bakery,Grocery Store,Italian Restaurant,Greek Restaurant,Steakhouse,Bagel Shop,Chinese Restaurant,Coffee Shop,Asian Restaurant
25,Douglaston,Grocery Store,Bakery,Greek Restaurant,Italian Restaurant,Korean Restaurant,Pizza Place,Asian Restaurant,Bagel Shop,Coffee Shop,Golf Course
29,Fresh Meadows,Pizza Place,Park,Bakery,Korean Restaurant,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Indian Restaurant,Grocery Store
32,Oakland Gardens,Pizza Place,Bakery,Park,Korean Restaurant,Italian Restaurant,Greek Restaurant,Gym,Bagel Shop,Indian Restaurant,Sushi Restaurant
45,Beechhurst,Pizza Place,Korean Restaurant,Park,Bakery,Deli / Bodega,Supermarket,Ice Cream Shop,Bagel Shop,Coffee Shop,Hotpot Restaurant


#### Queens Cluster 4

In [93]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,South Ozone Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Hollis,Pizza Place,Caribbean Restaurant,Park,Sandwich Place,Deli / Bodega,Gym,Sushi Restaurant,Café,Italian Restaurant,Restaurant
35,South Jamaica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,St. Albans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Rochdale,Caribbean Restaurant,Park,Airport Lounge,Bakery,Mexican Restaurant,Restaurant,Pizza Place,Deli / Bodega,Southern / Soul Food Restaurant,Gym
38,Springfield Gardens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Cambria Heights,Caribbean Restaurant,Pizza Place,Bakery,Italian Restaurant,Park,Pharmacy,Latin American Restaurant,Fast Food Restaurant,Convenience Store,Cosmetics Shop
40,Rosedale,Caribbean Restaurant,Pizza Place,Park,Italian Restaurant,Liquor Store,Sushi Restaurant,Mexican Restaurant,Clothing Store,Grocery Store,Fast Food Restaurant
59,Laurelton,Caribbean Restaurant,Park,Pizza Place,Liquor Store,Pharmacy,Latin American Restaurant,Airport Lounge,Fast Food Restaurant,Furniture / Home Store,Gym
64,Brookville,Caribbean Restaurant,Park,Pizza Place,Airport Lounge,Cosmetics Shop,Mexican Restaurant,Gym,Sandwich Place,American Restaurant,Bakery


#### Queens Cluster 5

In [94]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Bar,Pizza Place,Italian Restaurant,Thai Restaurant,Bagel Shop,Greek Restaurant,Coffee Shop,Butcher,Food Truck,Sushi Restaurant
1,Woodside,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jackson Heights,Thai Restaurant,Bakery,Pizza Place,Tennis Stadium,Italian Restaurant,Bagel Shop,Park,South American Restaurant,Sandwich Place,Martial Arts School
3,Elmhurst,Thai Restaurant,Pizza Place,Tennis Stadium,Bakery,Bagel Shop,Park,South American Restaurant,Sushi Restaurant,Italian Restaurant,Martial Arts School
5,Corona,Tennis Stadium,Thai Restaurant,South American Restaurant,Museum,Italian Restaurant,Park,Peruvian Restaurant,Pizza Place,Chinese Restaurant,Burger Joint
10,Long Island City,Park,Gym,Plaza,Bakery,Cocktail Bar,Sandwich Place,Pizza Place,Bar,Butcher,Ice Cream Shop
11,Sunnyside,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,East Elmhurst,Thai Restaurant,Bakery,Tennis Stadium,Pizza Place,Italian Restaurant,Latin American Restaurant,Food Truck,Indian Restaurant,Peruvian Restaurant,Hotpot Restaurant
13,Maspeth,Pizza Place,Bar,Brewery,Thai Restaurant,Italian Restaurant,Mexican Restaurant,Nightclub,Coffee Shop,South American Restaurant,Bakery
14,Ridgewood,Bar,Pizza Place,Bakery,Brewery,Music Venue,Nightclub,Coffee Shop,Mexican Restaurant,Cocktail Bar,Italian Restaurant
